# (Under construction) Euler's Disk, hydroelastic contact between a compliant cylinder and a compliant box
Damrong Guoy

From source installation of Drake, start this notebook by:
```
drake $ bazel run //tutorials:hydro_compliant_cylinder_compliant_box
```

This tutorial follows the pattern of [authoring_multibody_simulation.ipynb](./authoring_multibody_simulation.ipynb) by Zach Feng. If you are not familiar with Drake, you should study that one first.

# Introduction

This tutorial shows you how to set up simulations with hydroelastic contacts between two compliant hydroelastic geometries. It is motivated by [Euler's Disk](https://en.wikipedia.org/wiki/Euler%27s_Disk) invented by Joseph Bendik around 1990.

This tutorial is a sequel to the tutorial for *Hydroelastic Contact Between A Compliant Box And A Rigid Box*. You should study that tutorial before.

Click [here](./images/2023-03-31_EulerDisk_hydro_compliant_cylinder_box.html) to see playback from html record.

## Outline

This tutorial will show you how to:


## Start MeshCat

![StartMeshcat](./images/StartMeshcat.png)

In [1]:
from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Create compliant hydroelastic cylinder

*Make sure you have the MeshCat tab opened in your browser; the link is shown immediately above.*

We will create and visualize a compliant hydroelastic cylinder with SDFormat string.

We will use `ModelVisualizer` to verify the SDFormat description. It will tell MeshCat to show the cylinder.

Notice the three main blocks `<visual name="visual">`, `<collision name="collision">`, and `<inertial>` in the SDFormat string below. If you don't know them, the prequel tutorial has more explanation.

Our focus is in the `<collision>` block for contact simulation. Drake also uses the term proximity for collision. This is the `<drake:proximity_properties>` block that control hydroelastic contacts:

        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mesh_resolution_hint>0.01</drake:mesh_resolution_hint>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0.1</drake:relaxation_time>
        </drake:proximity_properties>

Use `<drake:compliant_hydroelastic/>` to specify a compliant object.

The `<drake:hydroelastic_modulus>1e7` (Pascals) indicates 10 MPa in hydroelastic modulus. It is comparable to high-density polyethylene (HDPE) with 1 GPa Young's modulus. Hydroelastic modulus is not a physical property but rather a numerical parameter to tune our contact model. As a rule of thumb, set hydroelastic modulus to about 1/100 of Young's modulus. 

The `<drake:mu_dynamic>0.5` (unitless) sets the friction coefficient.

The `<drake:hunt_crossley_dissipation>1.25` (seconds/meter) sets the dissipation constant for TAMSI contact solver, and the `<drake:relaxation_time>0.1` (seconds) sets the dissipation constant for SAP contact solver. Here we set both of them, so we can switch between the two solvers. In the future, we expect SAP to support Hunt & Crossley dissipation model. See *Hydroelastic contact* in [MultibodyPlant](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html) documentation.

We also need mass property in `<inertial>` block. The next code block shows you how to compute inertial matrix. We will use these numbers in our SDFormat string afterwards.

In [2]:
from pydrake.multibody.tree import UnitInertia

# Cylinder of radius 4 centimeters and height 1 centimeter
unit_inertia = UnitInertia.SolidCylinder(r=0.04, L=0.01)
# mass 1 kg
mass = 1

inertia_matrix3 = mass * unit_inertia.CopyToFullMatrix3()

print(inertia_matrix3)

[[0.00040833 0.         0.        ]
 [0.         0.00040833 0.        ]
 [0.         0.         0.0008    ]]


In [9]:
from pydrake.visualization import ModelVisualizer

# Define a compliant hydroelastic box.
compliant_cylinder_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantCylinder">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_cylinder_link">
      <inertial>
        <mass>1.0</mass>
        <inertia>
          <ixx>0.00040833</ixx> <ixy>0.0</ixy> <ixz>0.0</ixz>
          <iyy>0.00040833</iyy> <iyz>0.0</iyz>
          <izz>0.0008</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <cylinder>
            <radius>0.04</radius>
            <length>0.01</length>
          </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>
            <!-- 2 GPa for objects of size in centimeters of steel with Young's modulus 200 GPa -->
            2e9
          </drake:hydroelastic_modulus>
          <drake:mesh_resolution_hint>
            <!-- 5mm mesh resolution for objects of size in centimeters is for high accuracy -->
            0.005
          </drake:mesh_resolution_hint>
          <drake:mu_dynamic>1</drake:mu_dynamic>
          <drake:mu_static>1</drake:mu_static>
          <drake:hunt_crossley_dissipation>0</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0</drake:relaxation_time>
        </drake:proximity_properties>
      </collision>
      <visual name="EulerDisk">
        <geometry>
          <cylinder>
            <radius>0.04</radius>
            <length>0.01</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
      <visual name="InternalRod">
        <geometry>
          <box>
            <size>0.04 0.005 0.005</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 0.0 1.0</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>

"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(compliant_cylinder_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

## Create compliant hydroelastic box

The following SDFormat string specifies a rigid hydroelastic box. It is similar to the compliant hydroelastic box.

Both the `<visual>` and `<collision>` geometries are a box of size 30cm x 50cm x 3cm. 

This rigid hydroelastic box uses the same `<drake:proximity_properties>` block as the previous compliant hydroelastic box.

We do not specify `<mass>` and `<inertia>` of this box because we will fix it to the world frame when we are ready to create scene. It will not move.
    
The `<frame name="top_surface">` is a frame at the top of the box. It is at 2.5cm (half of the box's height) above the center of the box. In the next section *Create Diagram of the scene*, we will places the `top_surface` frame at the world's origin.

In [10]:
from pydrake.visualization import ModelVisualizer

# Create a compliant-hydroelastic table top
compliant_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <link name="compliant_box_link">
      <visual name="visual">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.3 0.5 0.03</size>
          </box>
        </geometry>
        <material>
         <diffuse>0.9 0.8 0.7 0.5</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.3 0.5 0.03</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>
            2e9
          </drake:hydroelastic_modulus>
          <drake:mu_dynamic>1</drake:mu_dynamic>
          <drake:mu_static>1</drake:mu_static>
          <drake:hunt_crossley_dissipation>0</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0</drake:relaxation_time>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="top_surface">
      <pose relative_to="compliant_box_link">0 0 0.025 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(compliant_box_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

## Create Diagram of the scene

The function `create_scene()` below creates a scene using the two objects that we created above.

It uses `DiagramBuilder` to create `MultibodyPlant` and `SceneGraph`. It uses `Parser` to add the two SDFormat strings of the two boxes into `Diagram` of the scene.

It fixes the rigid box's top surface to the world frame by calling `WeldFrames()`.

It sets the initial pose of the cylinder in the world frame.

After this step, the next section will add visualization to `DiagramBuilder`.

In [11]:
import numpy as np

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.framework import DiagramBuilder

def create_scene(time_step=1e-3, solver="tamsi"):
    # Clear MeshCat window from the previous blocks.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_solver=solver),
        builder)
    parser = Parser(plant)

    parser.AddModelsFromString(compliant_cylinder_sdf, "sdf")
    parser.AddModelsFromString(compliant_box_sdf, "sdf")

    # Weld the rigid box to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Initial pose of the cylinder
    # W = world frame
    # C = frame at the center of the compliant cylinder
    X_WC = RigidTransform(p=[0, 0, 0.03], rpy=RollPitchYaw(0.4 * np.pi, 0, 0))
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("compliant_cylinder_link"), X_WC)

    return builder, plant, scene_graph

## Set up visualization of the simulation

The function `create_scene()` above does not have visualization, so we will add visualization in the function `create_scene_and_viz()` below. 

We disable `publish_contacts` in this step.  We will add contact visualization in another section with more details.

In [12]:
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

def create_scene_and_viz(time_step=1e-3, solver="tamsi"):
    builder, plant, scene_graph = create_scene(time_step, solver)
   
    ApplyVisualizationConfig(
        config=VisualizationConfig(
                   publish_period = 1 / 512.0,
                   publish_contacts = False),
        builder=builder, meshcat=meshcat)
    
    return builder, plant


from pydrake.systems.analysis import Simulator

# Test creation of the diagram by simulating for 0 second.
# For now, use only the DiagramBuilder from the first return value and
# ignore the other two return values. We will use them later.
builder = create_scene_and_viz()[0]
simulator = Simulator(builder.Build())
simulator.Initialize()
simulator.AdvanceTo(0)

# Report contact results numerically

We will show you how to report contact results. We will create a simple system to read contact results from `MultibodyPlant` and print them at the end of simulation.  It is the same code as the prequel tutorial that explains how it works.

In [13]:
from pydrake.systems.framework import LeafSystem
from pydrake.common.value import AbstractValue
from pydrake.multibody.plant import ContactResults

class HydroelasticContactReporter(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self.DeclareAbstractInputPort(
            name="contact_results",
            model_value=AbstractValue.Make(
                # Input port will take ContactResults from MultibodyPlant
                ContactResults()))
        # Calling `ForcedPublish()` will trigger the callback.
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context):
        print()
        print(f"ContactReporter::Publish() called at time={context.get_time()}")
        contact_results = self.get_input_port().Eval(context)
        
        num_hydroelastic_contacts = contact_results.num_hydroelastic_contacts()
        print(f"num_hydroelastic_contacts() = {num_hydroelastic_contacts}")
        
        for c in range(num_hydroelastic_contacts):
            print()
            print(f"hydroelastic_contact_info({c}): {c}-th hydroelastic contact patch")
            hydroelastic_contact_info = contact_results.hydroelastic_contact_info(c)
            
            spatial_force = hydroelastic_contact_info.F_Ac_W()
            print(f"F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \\")
            print(f"{spatial_force}")
                        
            print(f"contact_surface()")
            contact_surface = hydroelastic_contact_info.contact_surface()
            num_faces = contact_surface.num_faces()
            total_area = contact_surface.total_area()
            centroid = contact_surface.centroid()
            print(f"total_area(): area of contact surface in m^2 = {total_area}")
            print(f"num_faces(): number of polygons or triangles = {num_faces}")
            print(f"centroid(): centroid (in World frame) = {centroid}")        
        
        print()

def create_scene_with_contact_report(time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_and_viz(time_step, solver)
    
    contact_reporter = builder.AddSystem(HydroelasticContactReporter())    
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_reporter.get_input_port(0))
        
    return builder, plant

# Visualize contact results

To visualize contact results, we will add `ContactVisualizer` to `Diagram` of the previous section.

The following function `create_scene_with_contact_report_and_viz()` adds `ContactVisualizer` to `DiagramBuilder` using `MultibodyPlant` and `meshcat`.

With `newtons_per_meter= 1e2`, it will draw a red arrow of length 1 centimeter for each force of 1 newton. With `newtons_meters_per_meter= 1e0`, it will draw a blue arrow of length 1 meter for each torque of 1 newton\*meter.

In [14]:
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams


def create_scene_with_contact_report_and_viz(time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_with_contact_report(time_step, solver)
    
    ContactVisualizer.AddToBuilder(
        builder, plant, meshcat,
        ContactVisualizerParams(
            publish_period= 1.0 / 512.0,
            newtons_per_meter= 1e2,
            newton_meters_per_meter= 5e-1))

    return builder, plant

## Run simulation with contact visualization

The following code will run the simulation and show pictures like this:

![EulerDisk_force_torque](./images/EulerDisk_force_torque.png)

The red arrow represents the force `f`, and the blue arrow represents the torque `tau`, which is also reported numerically at the end of simulation.

We set `simulator.set_target_realtime_rate(1)`, so `simulator` will try to publish results at real-time rate. However, since we will use small time step to capture physics, it is unlikely to be real time. The next section *Playback the recording* will replay animation, and then you can choose the playback speed.

At the end of simulation, the contact report like `f=[-6.889110099915437e-13, -1.5287151408153817e-11, 9.810000000350467]` means the force acting on the cylinder is 9.81 newtons, which makes sense because the cylinder is 1 kg. The documentation of [ContactResults](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_contact_results.html) and [HydroelasticContactInfo](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_hydroelastic_contact_info.html) has more explanation.

In [15]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_report_and_viz(sim_time, time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_with_contact_report_and_viz(time_step, solver)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1)
    
    v_init = np.hstack((np.array([0, 0, 2 * np.pi]), np.array([0, 0, 0])))
    
    plant_context = diagram.GetSubsystemContext(plant, simulator.get_context())
    plant.SetVelocities(plant_context, v_init)
    
    meshcat.StartRecording(frames_per_second=512.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    # Numerically report contact results at the end of simulation.
    diagram.ForcedPublish(simulator.get_context())

    
run_simulation_with_contact_report_and_viz(sim_time=3, time_step=1e-4)


ContactReporter::Publish() called at time=3.0
num_hydroelastic_contacts() = 1

hydroelastic_contact_info(0): 0-th hydroelastic contact patch
F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \
SpatialForce(
  tau=[-5.098176791024853e-13, -7.577400020474465e-14, 6.077687608545623e-19],
  f=[-6.889110099915437e-13, -1.5287151408153817e-11, 9.810000000350467],
)
contact_surface()
total_area(): area of contact surface in m^2 = 0.0050138449172460195
num_faces(): number of polygons or triangles = 343
centroid(): centroid (in World frame) = [-0.02448112 -0.00362292 -0.01000001]



## Playback the recording

In MeshCat tab, playback with `timeScale` 1, 0.1, and 0.05 to appreciate dynamics. You should see the force and torque vectors oscillate synchronously with the rolling disk.

If you are not familiar with `Animations` in MeshCat, please see the prequel tutorial.

In [16]:
# Hack to delete the contact surface because MeshcatAnimation does not record it.
meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_cylinder_link+compliant_box_link/contact_surface");

meshcat.PublishRecording()

# Extra 1

Uncomment the code below and run it. How large is force and torque at 0.2 second?

In [ ]:
# run_simulation_with_contact_report_and_viz(sim_time=0.2, time_step=1e-4)

# Extra 2

Uncomment the code below and run it to record results into a html file.

In [ ]:
# html_file = open("/home/damrongguoy/record_hydro_compliant_cylinder_compliant_box.html", "wt")
# html_file.write(meshcat.StaticHtml())
# html_file.close()

## Further reading

* [hydroelastic contact user guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)